In [3]:
import numpy as np
from numdifftools.core import Gradient, Hessian
import matplotlib.pyplot as plt
import decimal
from decimal import Decimal

<h3>Problem 1:</h3>
$$f(x_1, x_2)=100(x_2-x_1^2)^2+(1-x_1)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
-400x_1(x_2-x_1^2)-2(1-x_1)\\ 
200(x_2-x_1^2)\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
1200x_1^2-400x_2+2 & -400x_1\\ 
-400x_1 & 200\\ 
\end{pmatrix}$$
starting points $x_0$:
$$(1.2, 1.2), (-1.2, 1), (0.2, 0.8)$$
with the minimum at $x^*=(1,1)$

In [4]:
def fct_1(x: np.array) -> int:
    return 100*(x[1]-x[0]**2)**2 + (1-x[0])**2

def grad_fct_1(x: np.array) -> np.array:
    return np.array([-400*x[0]*(x[1]-x[0]**2)-2*(1-x[0]),
                    200*(x[1]-x[0]**2)], dtype=np.float64)

def hessian_fct_1(x: np.array) -> np.array:
    return np.array([[1200*x[0]**2-400*x[1]+2, -400*x[0]], 
                     [-400*x[0], 200]], dtype=np.float64)

<h3>Problem 2:</h3>
$$f(x_1, x_2)=150(x_1 x_2)^2+(0.5x_1 + 2x_2 - 2)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
300x_2^2 x_1 + 0.5 x_1 + 2x_2 - 2\\ 
300x_1^2 x_2 + 2x_1 + 8x_2 - 8\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
300x_2^2 +0.5 & 600x_1 x_2+2\\ 
600x_1 x_2+2 & 300x_1^2 +8\\ 
\end{pmatrix}$$

starting points $x_0$:
$$(-0.2, 1.2), (3.8, 0.1), (1.9, 0.6)$$<br>
with the minimums at $x^*=(0,1)$ and $x^*=(4,0)$ and a saddle point at $(0.43685, 0.10921)$

In [5]:
def fct_2(x: np.array) -> int:
    return 150*(x[0] * x[1])**2 + (0.5 * x[0] + 2*x[1] - 2)**2

def fct_2_dec(x: np.array) -> int:
    return 150*(Decimal(x[0]) * Decimal(x[1]))**2 + (Decimal(0.5) * Decimal(x[0]) + 2*Decimal(x[1]) - 2)**2

def grad_fct_2(x: np.array) -> np.array:
    return np.array([300*x[0]*x[1]**2 + 0.5*x[0]+2*x[1]-2,
                     300*(x[0]**2)*x[1] + 2*x[0]+8*x[1]-8], dtype=np.float64)

def hessian_fct_2(x: np.array) -> np.array:
    return np.array([[300*x[1]**2 + 0.5, 600*x[0]*x[1] + 2], 
                     [600*x[0]*x[1] + 2, 300*x[0]**2 + 8]], dtype=np.float64)

<h2>1. Implementation of NM with hessian modification</h2>

In [6]:
def Backtrack(f, x, gradient, pk, alpha_zero=1, rho=0.5, c=0.5):
    alpha = alpha_zero

    while not f(x + alpha * pk) <= float(f(x)) + c * alpha * np.dot(gradient.T, pk):
        alpha *= rho
        
    return alpha

In [7]:
def is_pos_def(A):
    try:
        np.linalg.cholesky(A)
        return True
    except np.linalg.LinAlgError:
        return False

In [8]:
def cholesky(a):
    iters = 100000
    i = 0
    beta = 1
    m = np.amin(np.diag(a))
    if m > 0:
        t = 0
    else:
        t = beta - m
    
    while i < iters:
        #print(a+np.dot(t,np.eye(a.shape[0])))
        try:
            l = np.linalg.cholesky(a+np.dot(t,np.eye(a.shape[0])))
            return l
        except np.linalg.LinAlgError:
            if 2*t > beta:
                t = 2*t
            else:
                t = beta
            i += 1
    return 'fail'

In [19]:
def newton_method(f, grad, hess, x0, max_iter=10000, tol=1e-6, c=0.05, rho=0.5, estimate=False):
    
    x = x0
    num_iter = 0
    e_g = Decimal(0.0000000000001)
    e_h = Decimal(0.00000001)
    
    while num_iter < max_iter:
        if estimate:
            x_dec = np.array([Decimal(val) for val in x])
            gradient = grad(f,x_dec,e_g)
            hessian = hess(f,x_dec,e_h,e_g)
            #print(gradient)
        else:  
            gradient = grad(x)
            hessian = hess(x) 
            #print(x,gradient)
        
        grad_norm = np.linalg.norm(gradient)
        if grad_norm < tol:
            break
                
        a=np.atleast_2d(hessian)
        #print(a)
        if not is_pos_def(a): #check if hessian is pos. definite, else make it pos. definite
            a = cholesky(a)
            #print('a')
        b=np.atleast_1d(gradient)
        search_direction = np.linalg.solve(a, -b)#[0]
        #print(search_direction)
        
        alpha = Backtrack(f=f, x=x, gradient=gradient, pk=search_direction, alpha_zero=1, rho=rho, c=c)
        #print(alpha)
        
        #print(x,gradient,hessian,alpha)
        x = x + alpha * search_direction
        num_iter += 1
        #print(x)
    x_star = x
    f_star = f(x_star)
    return x_star, round(float(f_star),15), num_iter, grad_norm

<h2>3. Gradient and hessian approximation</h2>

In [20]:
# use to get gradient as np.array
def grad_estimate_np(f, x: np.array, eps: decimal.Decimal):
    grad=np.full(len(x), Decimal(0))
    for i in range(len(x)):
        unit_vector = np.full(len(x), Decimal(0))
        unit_vector[i] = Decimal(1)
        grad[i] = round(float((f(x + (eps * unit_vector)) - f(x)) / eps), 15)
    return np.array(grad, dtype=float)

# use for further calculation of hessian estimate
def grad_estimate(f, x: np.array, eps: decimal.Decimal):
    grad=np.full(len(x), Decimal(0))
    for i in range(len(x)):
        unit_vector = np.full(len(x), Decimal(0))
        unit_vector[i] = Decimal(1)
        grad[i] = (f(x + (eps * unit_vector)) - f(x)) / eps
    return np.array(grad)

In [21]:
def hessian_estimate(f, x: np.array, eps_hess: decimal.Decimal, eps_grad):
    hessian = np.full((len(x), len(x)), Decimal(0))
    for i in range(len(x)):
        unit_vector = np.full(len(x), Decimal(0))
        unit_vector[i] = Decimal(1)
        hessian[:, i] = np.array([round(float(g),15) for g in (np.divide(grad_estimate(f=f, x=(x + (eps_hess * unit_vector)), eps=eps_grad) - grad_estimate(f=f, x=x, eps=eps_grad), eps_hess))])
    return np.array(hessian, dtype=float)

<h2>4. Problems to test NM with hessian modification using exact derivatives</h2>

<h3>Problem 1.1:</h3>
$$f(x_1, x_2)=100(x_2-x_1^2)^2+(1-x_1)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
-400x_1(x_2-x_1^2)-2(1-x_1)\\ 
200(x_2-x_1^2)\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
1200x_1^2-400x_2+2 & -400x_1\\ 
-400x_1 & 200\\ 
\end{pmatrix}$$
on the starting points $x_0$:
$$(1.2, 1.2)$$

<h4>using exact gradient and hessian:</h4>

In [28]:
x_0= np.array([1.2, 1.2])
x_true = [1, 1]
x_star, fval, it, grad_norm = newton_method(fct_1, grad_fct_1, hessian_fct_1, x_0, max_iter=100)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:',np.linalg.norm(x_true - x_star))

minimum 0.0  at x = [1. 1.] after 8    iterations with remaining gradient norm 1.4360392201267428e-11
distance to true solution: 1.7227828323731315e-13


<h4>using approximated gradient and hessian:</h4>

In [29]:
x_star, fval, it, grad_norm = newton_method(fct_1, grad_estimate_np, hessian_estimate, x_0, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:',np.linalg.norm(x_true - x_star))

minimum 0.0  at x = [1. 1.] after 8    iterations with remaining gradient norm 1.6428426735387657e-11
distance to true solution: 6.547406849965465e-11


<h3>Problem 1.2:</h3>
$$f(x_1, x_2)=100(x_2-x_1^2)^2+(1-x_1)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
-400x_1(x_2-x_1^2)-2(1-x_1)\\ 
200(x_2-x_1^2)\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
1200x_1^2-400x_2+2 & -400x_1\\ 
-400x_1 & 200\\ 
\end{pmatrix}$$
on the starting points $x_0$:
$$(-1.2, 1)$$

<h4>using exact gradient and hessian:</h4>

In [30]:
x_0= np.array([-1.2, 1])
x_star, fval, it, grad_norm = newton_method(fct_1, grad_fct_1, hessian_fct_1, x_0, max_iter=100)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true - x_star))

minimum 0.0  at x = [1. 1.] after 21   iterations with remaining gradient norm 1.2166683130616293e-10
distance to true solution: 1.2165119381179673e-12


<h4>using approximated gradient and hessian:</h4>

In [31]:
x_star, fval, it, grad_norm = newton_method(fct_1, grad_estimate_np, hessian_estimate, x_0, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true - x_star))

minimum 0.0  at x = [1. 1.] after 21   iterations with remaining gradient norm 1.204456775895258e-10
distance to true solution: 7.327427879491117e-11


<h3>Problem 1.3:</h3>
$$f(x_1, x_2)=100(x_2-x_1^2)^2+(1-x_1)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
-400x_1(x_2-x_1^2)-2(1-x_1)\\ 
200(x_2-x_1^2)\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
1200x_1^2-400x_2+2 & -400x_1\\ 
-400x_1 & 200\\ 
\end{pmatrix}$$
on the starting points $x_0$:
$$(0.2, 0.8)$$

<h4>using exact gradient and hessian:</h4>

In [32]:
x_0= np.array([0.2, 0.8])
x_star, fval, it, grad_norm = newton_method(fct_1, grad_fct_1, hessian_fct_1, x_0, max_iter=100)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true - x_star))

minimum 0.0  at x = [1. 1.] after 10   iterations with remaining gradient norm 5.699507176150195e-08
distance to true solution: 4.5220709297918025e-09


<h4>using approximated gradient and hessian:</h4>

In [33]:
x_star, fval, it, grad_norm = newton_method(fct_1, grad_estimate_np, hessian_estimate, x_0, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true - x_star))

minimum 0.0  at x = [1. 1.] after 10   iterations with remaining gradient norm 5.69051174865548e-08
distance to true solution: 4.695287589444588e-09


<h3>Problem 2.1:</h3>
$$f(x_1, x_2)=150(x_1 x_2)^2+(0.5x_1 + 2x_2 - 2)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
300x_2^2 x_1 + 0.5 x_1 + 2x_2 - 2\\ 
300x_1^2 x_2 + 2x_1 + 8x_2 - 8\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
300x_2^2 +0.5 & 600x_1 x_2+2\\ 
600x_1 x_2+2 & 300x_1^2 +8\\ 
\end{pmatrix}$$

on the starting points $x_0$:
$$(-0.2, 1.2)$$
is close to the soltion $x^*=(0,1)$:

<h4>using exact gradient and hessian:</h4>

In [34]:
x_0= np.array([-0.2, 1.2])
x_true_0 = [0,1]
x_true_1 = [4,0]
x_star, fval, it, grad_norm = newton_method(fct_2, grad_fct_2, hessian_fct_2, x_0, max_iter=100)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true_0 - x_star))

minimum 0.0  at x = [-4.71275675e-14  1.00000000e+00] after 6    iterations with remaining gradient norm 1.4028846968253339e-11
distance to true solution: 8.470831034427952e-14


<h4>using approximated gradient and hessian:</h4>

In [35]:
x_star, fval, it, grad_norm = newton_method(fct_2_dec, grad_estimate_np, hessian_estimate, x_0, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true_0 - x_star))

minimum 0.0  at x = [-9.6773873e-14  1.0000000e+00] after 6    iterations with remaining gradient norm 1.4024368078455443e-11
distance to true solution: 9.843101526244279e-14


<h3>Problem 2.2:</h3>
$$f(x_1, x_2)=150(x_1 x_2)^2+(0.5x_1 + 2x_2 - 2)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
300x_2^2 x_1 + 0.5 x_1 + 2x_2 - 2\\ 
300x_1^2 x_2 + 2x_1 + 8x_2 - 8\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
300x_2^2 +0.5 & 600x_1 x_2+2\\ 
600x_1 x_2+2 & 300x_1^2 +8\\ 
\end{pmatrix}$$

on the starting points $x_0$:
$$(3.8, 0.1)$$
is close to the soltion $x^*=(4,0)$:

<h4>using exact gradient and hessian:</h4>

In [36]:
x_0= np.array([3.8, 0.1])
x_star, fval, it, grad_norm = newton_method(fct_2, grad_fct_2, hessian_fct_2, x_0, max_iter=100)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true_1 - x_star))

minimum 0.0  at x = [ 4.00000000e+00 -4.81302457e-17] after 6    iterations with remaining gradient norm 2.238280106885106e-13
distance to true solution: 3.553039685127206e-15


<h4>using approximated gradient and hessian:</h4>

In [37]:
x_star, fval, it, grad_norm = newton_method(fct_2_dec, grad_estimate_np, hessian_estimate, x_0, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true_1 - x_star))

minimum 0.0  at x = [ 4.00000000e+00 -5.00971536e-14] after 6    iterations with remaining gradient norm 2.2443707358633958e-13
distance to true solution: 1.3158972271918034e-13


<h3>Problem 2.3:</h3>
$$f(x_1, x_2)=150(x_1 x_2)^2+(0.5x_1 + 2x_2 - 2)^2$$
$$f'(x_1, x_2)=\begin{pmatrix}
300x_2^2 x_1 + 0.5 x_1 + 2x_2 - 2\\ 
300x_1^2 x_2 + 2x_1 + 8x_2 - 8\\ 
\end{pmatrix}$$
$$f''(x_1, x_2)=\begin{pmatrix}
300x_2^2 +0.5 & 600x_1 x_2+2\\ 
600x_1 x_2+2 & 300x_1^2 +8\\ 
\end{pmatrix}$$

on the starting points $x_0$:
$$(1.9, 0.6)$$
close $x^*=(0,1)$

<h4>using exact gradient and hessian:</h4>

In [44]:
x_0= np.array([1.9, 0.6])
x_star, fval, it, grad_norm = newton_method(fct_2, grad_fct_2, hessian_fct_2, x_0, max_iter=100)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true_0 - x_star))

minimum 1e-15 at x = [2.30536033e-09 1.00000001e+00] after 7    iterations with remaining gradient norm 7.054646907291285e-07
distance to true solution: 5.973414823268371e-09


<h4>using approximated gradient and hessian:</h4>

In [43]:
x_star, fval, it, grad_norm = newton_method(fct_2_dec, grad_estimate_np, hessian_estimate, x_0, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')
print('distance to true solution:', np.linalg.norm(x_true_0 - x_star))

minimum 1e-15 at x = [2.30487599e-09 1.00000001e+00] after 7    iterations with remaining gradient norm 7.053436877426599e-07
distance to true solution: 5.976575094501265e-09
